<h1>User Engagement Analysis</h1>

In [1]:
# Importing the neccesary libraries and packages

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append("..")

In [38]:
# Reading the data 

engagement_data = pd.read_csv('../data/user_overview_data.csv')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [39]:
# Rename the columns for simplicity

engagement_data.rename(columns={'Bearer Id':'Session Frequency', 'Dur. (ms)':'Total Duration', 'Total_UL_and_DL':'Total traffic'}, inplace= True)

In [40]:
# Grouping and aggregating the data

engagement_data = engagement_data.groupby('MSISDN/Number')
engagement_data = engagement_data.agg({'Session Frequency':'count','Total Duration':'sum','Total traffic':'sum'})

In [ ]:
# Creating a data set containing only the specified metrics

engagement_cols = ['Session Frequency', 'Total Duration', 'Total traffic']
engagement_agg = engagement_data[engagement_cols]